# Creacion Consolidado por ROL

Comenzaremos el analisis de la data consolidando la info en un unico DataSet, para ello vamos a considerar los siguientes puntos:


- DATA:  Se considerará toda las causas que ingresaron al Sistema desde 2015 al 2019, esto ya que se cuenta además con la data de las audiencias en ese período.


- No se consideran causas anteriores a este período.

- Se considerarán Causas sin termino.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from src.data import cleandata
from src.data import transformdata

In [2]:
warnings.filterwarnings(action="ignore")
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
path_processed = "../data/processed/pjud"

#### Paso 1 : Consolidacion de Dataset x rol

In [5]:
df_ingresos_rol = pd.read_feather(f"{path_processed}/IngresosRol.feather")
print(f"{len(df_ingresos_rol)} causas en el dataset de ingresos por Rol")

2980483 causas en el dataset de ingresos por Rol


In [6]:
df_termino_rol = pd.read_feather(f"{path_processed}/TerminoRol.feather")
print(f"Existen : {len(df_termino_rol)} causas en el dataset de termino por rol")

Existen : 2945536 causas en el dataset de termino por rol


Analizamos la data, uniendo ambos dataset.

In [7]:
df_fulldata_rol = pd.merge(df_ingresos_rol, df_termino_rol, how='outer', on=['COD. TRIBUNAL','RIT'])
print(f"Existen: {len(df_fulldata_rol)} causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso")

Existen: 3894215 causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso


### Analisis de la Data Causas por Rol

De estos dos dataset debo considerar:

- Caso 1: el dataset df_causas_terminadas completo
- Caso 2: Debo agregar además las causas que tengan Fecha Ingreso_x pero que aún no terminan.
- Caso 3: Debo analizar las causas que no tienen Fecha Ingreso_x pero si tienen Fecha Ingreso_y y Fecha Ingreso_y, ya que deben ser las causas que no fueron tomadas por la consulta de Ingresos. Acá se debe reemplazar los datos obtenidos en Termino Rol a Ingresos.

Luego de tener estos tres dataset debo concatenarlos

In [8]:
df_fulldata_rol

,level_0_x,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,FECHA INGRESO_x,MES INGRESO,AÑO INGRESO,level_0_y,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,FECHA INGRESO_y,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
0,0.0,0.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,1-2018,Ordinaria,2018-01-02,ene-18,2018.0,21.0,21.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-02,2018-03-05,62.0,SENTENCIA,mar-18,2018.0,1.0
1,1.0,1.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,2-2018,Ordinaria,2018-01-03,ene-18,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2,2.0,2.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,3-2018,Ordinaria,2018-01-03,ene-18,2018.0,223.0,223.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-02-20,48.0,SENTENCIA,feb-18,2018.0,1.0
3,3.0,3.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,4-2018,Ordinaria,2018-01-03,ene-18,2018.0,335.0,335.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-03-26,82.0,SENTENCIA,mar-18,2018.0,1.0
4,4.0,4.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,5-2018,Ordinaria,2018-01-03,ene-18,2018.0,426.0,426.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,2018-01-03,2018-04-18,105.0,SENTENCIA,abr-18,2018.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894210,NaN,NaN,NaN,NaN,1226,NaN,0-2015,NaN,NaT,NaN,NaN,3171671.0,494794.0,90.0,C.A. DE SANTIAGO,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2015-12-02,2015-12-03,1.0,DECLARA INCOMPETENCIA.,dic-15,2015.0,1.0
3894211,NaN,NaN,NaN,NaN,1226,NaN,0-2015,NaN,NaT,NaN,NaN,3171672.0,494795.0,90.0,C.A. DE SANTIAGO,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2015-12-22,2015-12-31,9.0,DECLARA INCOMPETENCIA.,dic-15,2015.0,1.0
3894212,NaN,NaN,NaN,NaN,1232,NaN,0-2015,NaN,NaT,NaN,NaN,3230594.0,553717.0,90.0,C.A. DE SANTIAGO,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2015-10-04,2015-10-14,10.0,CERTIFICA CUMPLIMIENTO ART. 468.,oct-15,2015.0,1.0
3894213,NaN,NaN,NaN,NaN,1232,NaN,0-2015,NaN,NaT,NaN,NaN,3230595.0,553718.0,90.0,C.A. DE SANTIAGO,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2015-11-10,2015-12-21,41.0,DECLARA INCOMPETENCIA.,dic-15,2015.0,1.0


In [9]:
df_fulldata_rol.columns

Index(['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL',
       'TRIBUNAL_x', 'RIT', 'TIPO CAUSA_x', 'FECHA INGRESO_x', 'MES INGRESO',
       'AÑO INGRESO', 'level_0_y', 'index_y', 'COD. CORTE_y', 'CORTE_y',
       'TRIBUNAL_y', 'TIPO CAUSA_y', 'FECHA INGRESO_y', 'FECHA TERMINO',
       'DURACION CAUSA', 'MOTIVO TERMINO', 'MES TERMINO', 'AÑO TERMINO',
       'TOTAL TERMINOS'],
      dtype='object')

In [10]:
columnas_drop = ['level_0_x', 'index_x', 'level_0_y', 'index_y', 'MES INGRESO', 'MES TERMINO']

df_fulldata_rol.drop(columnas_drop, axis = 'columns', inplace = True)

In [11]:
df_fulldata_rol = df_fulldata_rol.progress_apply(transformdata.faltantes_rol, axis=1)

100%|██████████| 3894215/3894215 [21:56<00:00, 2958.07it/s] 


In [12]:
columnas_drop = ['TIPO CAUSA_y', 'TRIBUNAL_y', 'COD. CORTE_y', 'CORTE_y', 'FECHA INGRESO_y']

df_fulldata_rol.drop(columnas_drop, axis = 'columns', inplace = True)

In [13]:
df_fulldata_rol.rename(columns = {'COD. CORTE_x':'COD. CORTE',
                                  'CORTE_x':'CORTE',
                                  'TRIBUNAL_x':'TRIBUNAL',
                                  'TIPO CAUSA_x':'TIPO CAUSA',
                                  'MATERIA_x':'MATERIA',
                                  'FECHA INGRESO_x':'FECHA INGRESO'
                                  }, inplace = True)

In [14]:
df_fulldata_rol

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,1-2018,Ordinaria,2018-01-02,2018.0,2018-03-05,62.0,SENTENCIA,2018.0,1.0
1,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,2-2018,Ordinaria,2018-01-03,2018.0,NaT,NaN,SIN TERMINO,NaN,NaN
2,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,3-2018,Ordinaria,2018-01-03,2018.0,2018-02-20,48.0,SENTENCIA,2018.0,1.0
3,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,4-2018,Ordinaria,2018-01-03,2018.0,2018-03-26,82.0,SENTENCIA,2018.0,1.0
4,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,5-2018,Ordinaria,2018-01-03,2018.0,2018-04-18,105.0,SENTENCIA,2018.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894210,90.0,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-12-02,2015.0,2015-12-03,1.0,DECLARA INCOMPETENCIA,2015.0,1.0
3894211,90.0,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-12-22,2015.0,2015-12-31,9.0,DECLARA INCOMPETENCIA,2015.0,1.0
3894212,90.0,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-10-04,2015.0,2015-10-14,10.0,CERTIFICA CUMPLIMIENTO ART 468,2015.0,1.0
3894213,90.0,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-11-10,2015.0,2015-12-21,41.0,DECLARA INCOMPETENCIA,2015.0,1.0


In [15]:
df_fulldata_rol['MOTIVO TERMINO'].value_counts()

SIN TERMINO                                         948678
DECLARA SOBRESEIMIENTO DEFINITIVO                   705067
APROBACION NO INICIO INVESTIGACION                  636328
SENTENCIA                                           518657
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    358149
ACOGE REQUERIMIENTO (MONITORIO)                     311696
NO PERSEVERAR EN EL PROCEDIMIENTO                   179015
DECLARA INCOMPETENCIA                                87308
CERTIFICA CUMPLIMIENTO ART 468                       66844
ACUMULACION                                          46431
DECLARA INCOMPETENCIA RPA                            19927
DECLARA INADMISIBILIDAD DE LA QUERELLA                9110
ABANDONO DE LA QUERELLA                               5285
DESISTIMIENTO QUERELLA                                1720
Name: MOTIVO TERMINO, dtype: int64

In [17]:
print(f"Existen: {len(df_fulldata_rol)} causas por rol")

Existen: 3894215 causas por rol


In [27]:
causas_top = df_fulldata_rol[df_fulldata_rol['TRIBUNAL'].str.contains('ORAL')]
causas_garantia = df_fulldata_rol[df_fulldata_rol['TRIBUNAL'].str.contains('GARANTIA')]


In [28]:
causas_top

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,1-2018,Ordinaria,2018-01-02,2018.0,2018-03-05,62.0,SENTENCIA,2018.0,1.0
1,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,2-2018,Ordinaria,2018-01-03,2018.0,NaT,NaN,SIN TERMINO,NaN,NaN
2,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,3-2018,Ordinaria,2018-01-03,2018.0,2018-02-20,48.0,SENTENCIA,2018.0,1.0
3,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,4-2018,Ordinaria,2018-01-03,2018.0,2018-03-26,82.0,SENTENCIA,2018.0,1.0
4,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,5-2018,Ordinaria,2018-01-03,2018.0,2018-04-18,105.0,SENTENCIA,2018.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3893915,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,95--2018,Ordinaria,2018-10-19,2018.0,2019-03-19,151.0,SENTENCIA,2019.0,1.0
3893916,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,95--2019,Ordinaria,2019-10-16,2019.0,2019-12-10,55.0,SENTENCIA,2019.0,1.0
3893917,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,96--2019,Ordinaria,2019-10-17,2019.0,2019-12-16,60.0,SENTENCIA,2019.0,1.0
3893918,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,98--2018,Ordinaria,2018-10-24,2018.0,2019-04-30,188.0,SENTENCIA,2019.0,1.0


In [29]:
causas_garantia

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
480,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,1-2018,Ordinaria,2018-01-01,2018.0,2018-06-18,168.0,SENTENCIA,2018.0,1.0
481,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,1-2018,Ordinaria,2018-01-01,2018.0,2018-09-10,252.0,DECLARA INCOMPETENCIA,2018.0,1.0
482,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,2-2018,Ordinaria,2018-01-01,2018.0,2018-07-12,192.0,NO PERSEVERAR EN EL PROCEDIMIENTO,2018.0,1.0
483,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,3-2018,Ordinaria,2018-01-02,2018.0,2018-05-17,135.0,NO PERSEVERAR EN EL PROCEDIMIENTO,2018.0,1.0
484,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,3-2018,Ordinaria,2018-01-02,2018.0,2018-07-19,198.0,NO PERSEVERAR EN EL PROCEDIMIENTO,2018.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894131,40.0,C.A. DE TALCA,961,JUZGADO DE GARANTIA CURICO,3354-2015,Ordinaria,2015-06-26,2015.0,2016-03-23,271.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2016.0,1.0
3894145,56.0,C.A. DE PUERTO MONTT,245,JUZGADO DE LETRAS Y GARANTIA CHAITEN,184-2015,Ordinaria,2015-11-12,2015.0,2016-05-20,190.0,APROBACION NO INICIO INVESTIGACION,2016.0,1.0
3894155,90.0,C.A. DE SANTIAGO,1222,TERCER JUZGADO DE GARANTIA SANTIAGO,8580-2015,Ordinaria,2015-12-31,2015.0,2016-04-18,109.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2016.0,1.0
3894156,90.0,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,3772-2016,Ordinaria,2015-12-04,2015.0,2016-10-14,315.0,SENTENCIA,2016.0,1.0


In [22]:
causas_top['MOTIVO TERMINO'].value_counts()

SENTENCIA                            61935
SIN TERMINO                          16608
DECLARA SOBRESEIMIENTO DEFINITIVO     1657
ACUMULACION                            639
DECLARA INCOMPETENCIA                  204
ABANDONO DE LA QUERELLA                 44
DECLARA INCOMPETENCIA RPA                3
Name: MOTIVO TERMINO, dtype: int64

In [30]:
causas_garantia['MOTIVO TERMINO'].value_counts()

SIN TERMINO                                         932070
DECLARA SOBRESEIMIENTO DEFINITIVO                   703407
APROBACION NO INICIO INVESTIGACION                  636327
SENTENCIA                                           456722
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    358149
ACOGE REQUERIMIENTO (MONITORIO)                     311696
NO PERSEVERAR EN EL PROCEDIMIENTO                   179014
DECLARA INCOMPETENCIA                                86782
CERTIFICA CUMPLIMIENTO ART 468                       66838
ACUMULACION                                          45792
DECLARA INCOMPETENCIA RPA                            19809
DECLARA INADMISIBILIDAD DE LA QUERELLA                9110
ABANDONO DE LA QUERELLA                               5241
DESISTIMIENTO QUERELLA                                1720
Name: MOTIVO TERMINO, dtype: int64

ATENCION:
Se detecta que existen RIT 0-AÑO, lo cual no existe en la realidad, estos RIT deberían corresponder a errores del Sistema o ajustes para Metas de Gestión. Para efecto de este analisis deberían ELIMINARSE.

In [32]:
df_rit_cero = df_fulldata_rol[df_fulldata_rol['RIT'].str.startswith("0-")]

In [33]:
df_rit_cero

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS


In [34]:
df_fulldata_rol.drop(df_rit_cero.index, axis=0, inplace=True)

## CARGA DATA A ARCHIVOS FUTHER

In [35]:
# Directorio donde se guardaran archivos feather
df_fulldata_rol.reset_index(inplace = True)
causas_top.reset_index(inplace = True)
causas_garantia.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldata_rol.to_feather(f'{path_processed}/Consolidado_Rol.feather')
causas_top.to_feather(f'{path_processed}/JuicioOralesRol.feather')
causas_garantia.to_feather(f'{path_processed}/CausasGarantiaRol.feather')